목적 : 상장 결측치 채우기
=====================

목표 : 기업수명주기별, 부실별, 정규성 확인
-------------------------    
- 정규적 -> 평균으로 결측치
- 비정규적 -> 중앙값으로 결측치

import modules

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
import statsmodels.api as sm

import platform
if platform.system() == "Darwin":           # MAC인 경우
    plt.rc('font', family ="AppleGothic")
else:
    plt.rc('font',family='Malgun Gothic')

#멧플로립에서 음수를 표시
plt.rc('axes',unicode_minus=False)

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')


import scipy.stats as stats
from scipy.stats import shapiro, pearsonr
from itertools import combinations
from sklearn.model_selection import train_test_split

환경설정

In [2]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

# pd.set_option('display.max_columns', None)
pd.reset_option('display.max_columns')

pd.set_option('display.max_seq_items', None)

pd.set_option('float_format', '{:.3f}'.format) # 소수점 3개까지 출력

load file

In [3]:
listed = pd.read_csv("../data/datasets/상장수정필요피처.csv")
# 영업년수와 상장년수가 있더라

In [4]:
listed.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본정상영업이익률',
       '매출액정상영업이익률', '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율',
       '비유동자산증가율', '매출액증가율', '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율',
       '부가가치율', '이윤분배율', '자본분배율', '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율',
       '경영자본회전률', '총자본회전률', '비유동자산회전률', '유동자산회전률', '영업년수', '상장년수', 'PER',
       'PBR', 'PCR', 'PSR', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율', '영업재무현금비율',
       '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

# train test split

In [5]:
X = listed.drop('부실판단', axis=1)
y = listed['부실판단']
# 데이터를 학습용과 테스트용으로 나눔
listed_X_train, listed_X_test, listed_y_train, listed_y_test = train_test_split(X, y, test_size= 1 / (np.exp(1) + 1) , random_state=911, stratify=y)

In [6]:
listed_X_train['부실판단'] = listed_y_train

In [7]:
len(X)

731

In [8]:
len(listed_X_train)

534

In [9]:
len(listed_X_test)

197

# describe

all

In [10]:
listed_X_train

,회사명,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
563,성우테크론(주),45300,2014,9,13,4,9,76,76,75,...,0.073,4.102,-0.373,-1.152,15.063,0,0,1,0,1.000
54,(주)동일기연,32960,2022,76,363,5,12,2011,1533,7,...,0.306,9.729,0.939,-0.494,-13.817,0,1,0,0,0.000
265,(주)엔지켐생명과학,183490,2020,-28,-34,-12,-56,1073,979,71,...,0.806,-0.095,0.994,0.310,-0.238,1,0,0,0,1.000
634,유엔젤(주),72130,2022,-2,-8,0,-1,745,730,13,...,0.463,17.670,0.395,-0.736,1.024,0,0,0,1,0.000
286,(주)오이솔루션,138080,2022,-1,-2,0,-1,255,158,27,...,0.414,-2.773,1.349,-1.201,1.217,0,0,0,1,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,한국정밀기계(주),101680,2014,11,21,3,9,383,148,34,...,0.189,0.278,1.044,-1.139,3.854,0,0,1,0,1.000
409,(주)코세스,89890,2015,7,15,6,5,58,40,754,...,0.199,-78.045,0.910,-6.593,-0.222,0,1,0,0,1.000
344,(주)인터로조,119610,2022,18,29,5,10,215,166,39,...,0.205,1.940,1.090,-0.420,1.527,0,0,1,0,0.000
273,(주)엠젠솔루션,32790,2014,4,9,3,1,99,89,242,...,0.201,-15.947,0.130,-0.339,-2.702,0,1,0,0,1.000


In [11]:
listed_X_train.describe()

,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,534.000,534.000,534.000,534.000,534.000,534.000,534.000,534.000,534.000,534.000,...,534.000,534.000,534.000,534.000,531.000,534.000,534.000,534.000,534.000,534.000
mean,138169.607,2019.702,9.307,1664.285,-0.176,-0.968,299.161,247.049,128.135,101.848,...,0.353,0.223,0.631,2.380,13.183,0.298,0.290,0.223,0.189,0.504
std,121484.287,3.015,47.127,26931.452,14.915,29.483,384.260,353.386,419.643,381.869,...,0.486,283.809,2.947,72.371,318.112,0.458,0.454,0.417,0.392,0.500
min,40.000,2014.000,-276.000,-5211.000,-255.000,-451.000,4.000,4.000,0.000,0.000,...,0.015,-5550.949,-47.302,-244.700,-301.491,0.000,0.000,0.000,0.000,0.000
25%,49952.500,2017.000,-10.000,-15.000,-5.000,-8.000,103.000,70.000,32.000,20.000,...,0.127,-0.927,0.421,-1.295,-1.876,0.000,0.000,0.000,0.000,0.000
50%,101205.000,2022.000,2.000,2.000,1.000,1.000,166.000,121.500,72.000,49.000,...,0.219,1.744,0.832,-0.388,-0.467,0.000,0.000,0.000,0.000,1.000
75%,214445.000,2022.000,18.000,35.000,6.000,9.750,319.000,260.500,128.000,100.000,...,0.410,4.151,1.053,0.270,0.221,1.000,1.000,0.000,0.000,1.000
max,900300.000,2022.000,259.000,611057.000,44.000,90.000,3542.000,3246.000,9039.000,8254.000,...,7.116,3404.076,15.017,1529.301,7211.514,1.000,1.000,1.000,1.000,1.000


In [12]:
listed_X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 534 entries, 563 to 340
Data columns (total 60 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   회사명                534 non-null    object 
 1   거래소코드              534 non-null    int64  
 2   회계년도               534 non-null    int64  
 3   CASH FLOW 대 부채비율   534 non-null    int64  
 4   CASH FLOW 대 차입금비율  534 non-null    int64  
 5   CASH FLOW 대 총자본비율  534 non-null    int64  
 6   CASH FLOW 대 매출액비율  534 non-null    int64  
 7   유동비율               534 non-null    int64  
 8   당좌비율               534 non-null    int64  
 9   부채비율               534 non-null    int64  
 10  유동부채비율             534 non-null    int64  
 11  차입금의존도             534 non-null    int64  
 12  순운전자본비율            534 non-null    int64  
 13  현금비율               534 non-null    int64  
 14  유동자산구성비율           534 non-null    int64  
 15  자기자본구성비율           534 non-null    int64  
 16  자기자본순이익률           534 non-nu

In [13]:
listed_X_train.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본정상영업이익률',
       '매출액정상영업이익률', '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율',
       '비유동자산증가율', '매출액증가율', '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율',
       '부가가치율', '이윤분배율', '자본분배율', '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율',
       '경영자본회전률', '총자본회전률', '비유동자산회전률', '유동자산회전률', '영업년수', '상장년수', 'PER',
       'PBR', 'PCR', 'PSR', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율', '영업재무현금비율',
       '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [14]:
listed_X_train.isna().sum().sort_values(ascending=False)

영업재무현금비율             3
회사명                  0
영업년수                 0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
상장년수                 0
거래소코드                0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

In [15]:
listed_X_train.drop(['부실판단', '도입기', '성장기', '성숙기', '쇠퇴기'], axis=1).isin([0]).sum().sort_values(ascending=False) / len(listed_X_train) * 100

총자본회전률              37.079
경영자본회전률             24.345
이윤분배율               15.356
자본분배율               15.169
비유동자산회전률             9.925
유동자산회전률              8.614
당좌자산회전률              6.554
차입금의존도               6.180
CASH FLOW 대 총자본비율    4.307
비유동자산증가율             2.622
CASH FLOW 대 차입금비율    2.622
총자본증가율               2.434
유형자산회전율              2.434
유형자산증가율              2.434
매출액증가율               2.434
유동자산증가율              2.434
현금비율                 1.873
CASH FLOW 대 매출액비율    1.498
CASH FLOW 대 부채비율     1.124
PBR                  0.936
순운전자본비율              0.936
PSR                  0.936
PCR                  0.936
PER                  0.936
유동부채비율               0.749
부채비율                 0.562
자기자본구성비율             0.562
CROE                 0.375
자기자본순이익률             0.375
총자본정상영업이익률           0.187
총자본투자효율              0.187
부가가치율                0.187
매출채권회전률              0.187
영업레버리지               0.000
상장년수                 0.000
재무레버리지               0.000
쭈피처                  0.000
영

In [16]:
listed_X_train.isin([np.inf]).sum().sort_values(ascending=False)

회사명                  0
거래소코드                0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

In [17]:
listed_X_train.isin([-np.inf]).sum().sort_values(ascending=False)

회사명                  0
거래소코드                0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

In [18]:
# inf_col = unlisted_X_train.isin([np.inf]).sum().sort_values(ascending=False).index[0]
# max_value = unlisted_X_train[inf_col].replace(np.inf, np.nan).max()
# unlisted_X_train[inf_col] = unlisted_X_train[inf_col].replace(np.inf, max_value)

# 범주 분할 1현금흐름 / 2안정성 / 2수익성 / 3유동성 / 4성장성 / 5생산성 / 6활동성 / 7시장지표 및 기타

In [19]:
# 비상장 all data는 unlisted
# 비상장 label들

labels = listed_X_train[
    [
        '거래소코드',
        '회계년도',
        '부실판단',
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기'
    ]
]

# 비상장 피처
features = listed_X_train.drop(
    labels=[
        '회사명',
        '거래소코드',
        '회계년도',        
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'        
    ],
    axis=1,    
)

# 1현금흐름 4개
cash_ft1 = listed_X_train[
    [
        'CASH FLOW 대 부채비율',
        'CASH FLOW 대 차입금비율',
        'CASH FLOW 대 총자본비율',
        'CASH FLOW 대 매출액비율',
        '부실판단'
    ]
]

# 2안정성 9개
stability_ft1 = listed_X_train[
    [
        '유동비율',
        '당좌비율',
        '부채비율',
        '유동부채비율',
        '차입금의존도',
        '순운전자본비율',
        '현금비율',
        '유동자산구성비율',
        '자기자본구성비율',
        '부실판단'
    ]
]

# 3수익성 8개
profitability_ft1 = listed_X_train[
    [
        '자기자본순이익률',
        '경영자본순이익률',
        '매출액순이익률',
        '총자본사업이익률',
        '총자본정상영업이익률',
        '매출액정상영업이익률',
        '금융비용부담률',
        'CROE',
        '부실판단'
    ]
]

# 4성장성 6개
growth_ft1 = listed_X_train[
    [
        '총자산증가율',
        '총자본증가율',
        '유동자산증가율',
        '비유동자산증가율',
        '매출액증가율',
        '유형자산증가율',
        '부실판단'
    ]
]

# 5생산성 6개
productivity_ft1 = listed_X_train[
    [
        '부가가치(백만원)',
        '총자본투자효율',
        '설비투자효율',
        '부가가치율',
        '이윤분배율',
        '자본분배율',
        '부실판단'
    ]
]

# 6활동성 8개
activity_ft1 = listed_X_train[
    [
        '총자산회전율',
        '매출채권회전률',
        '당좌자산회전률',
        '유형자산회전율',
        '경영자본회전률',
        '총자본회전률',
        '비유동자산회전률',
        '유동자산회전률',
        '부실판단'
    ]
]

# 7기타
etc_ft1 = listed_X_train[
    [
        '영업년수',
        '쭈피처',
        '영업레버리지',
        '재무레버리지',
        '영업투자현금비율',
        '영업재무현금비율', 
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'
    ]
]

In [20]:
listed_X_train.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본정상영업이익률',
       '매출액정상영업이익률', '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율',
       '비유동자산증가율', '매출액증가율', '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율',
       '부가가치율', '이윤분배율', '자본분배율', '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율',
       '경영자본회전률', '총자본회전률', '비유동자산회전률', '유동자산회전률', '영업년수', '상장년수', 'PER',
       'PBR', 'PCR', 'PSR', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율', '영업재무현금비율',
       '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

1현금흐름 4가지 cash

In [21]:
cash_ft1.describe()

,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,부실판단
count,534.000,534.000,534.000,534.000,534.000
mean,9.307,1664.285,-0.176,-0.968,0.504
std,47.127,26931.452,14.915,29.483,0.500
min,-276.000,-5211.000,-255.000,-451.000,0.000
25%,-10.000,-15.000,-5.000,-8.000,0.000
50%,2.000,2.000,1.000,1.000,1.000
75%,18.000,35.000,6.000,9.750,1.000
max,259.000,611057.000,44.000,90.000,1.000


2안정성 9가지 stability

In [22]:
stability_ft1.describe()

,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,부실판단
count,534.000,534.000,534.000,534.000,534.000,534.000,534.000,534.000,534.000,534.000
mean,299.161,247.049,128.135,101.848,24.283,20.483,73.451,56.045,58.133,0.504
std,384.260,353.386,419.643,381.869,18.086,28.768,154.410,94.698,21.437,0.500
min,4.000,4.000,0.000,0.000,0.000,-90.000,0.000,3.000,0.000,0.000
25%,103.000,70.000,32.000,20.000,9.000,2.000,14.000,39.000,43.250,0.000
50%,166.000,121.500,72.000,49.000,23.000,19.500,30.000,51.000,57.500,1.000
75%,319.000,260.500,128.000,100.000,37.000,39.750,75.000,65.000,75.750,1.000
max,3542.000,3246.000,9039.000,8254.000,151.000,186.000,1981.000,2198.000,96.000,1.000


3수익성 8가지 profitability

In [23]:
productivity_ft1.describe()

,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,534.000,534.000,534.000,534.000,534.000,534.000,534.000
mean,16332.693,12.268,-127.650,19.770,-64.000,-19.573,0.504
std,25344.103,24.878,6502.675,40.874,267.864,187.620,0.500
min,-56429.000,-411.000,-145764.000,-345.000,-2834.000,-2061.000,0.000
25%,3615.750,5.000,18.000,10.000,-38.000,0.000,0.000
50%,10718.500,13.000,51.500,21.000,0.000,21.000,1.000
75%,23787.000,22.000,114.750,36.750,26.000,45.750,1.000
max,260543.000,99.000,25992.000,188.000,292.000,93.000,1.000


4성장성 6개 growth

In [24]:
growth_ft1.describe()

,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부실판단
count,534.000,534.000,534.000,534.000,534.000,534.000,534.000
mean,9.041,8.609,8.737,17.879,15.207,97.658,0.504
std,37.323,36.726,49.431,85.872,74.041,1759.522,0.500
min,-86.924,-80.960,-97.710,-91.420,-95.700,-99.670,0.000
25%,-6.210,-5.885,-13.680,-7.522,-10.820,-6.730,0.000
50%,3.262,2.430,0.810,0.985,4.640,0.000,1.000
75%,17.066,16.050,19.525,20.078,24.170,15.670,1.000
max,419.817,419.820,479.050,1160.470,1031.410,40598.090,1.000


5생산성 6가지 productivity

In [25]:
productivity_ft1.describe()

,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,534.000,534.000,534.000,534.000,534.000,534.000,534.000
mean,16332.693,12.268,-127.650,19.770,-64.000,-19.573,0.504
std,25344.103,24.878,6502.675,40.874,267.864,187.620,0.500
min,-56429.000,-411.000,-145764.000,-345.000,-2834.000,-2061.000,0.000
25%,3615.750,5.000,18.000,10.000,-38.000,0.000,0.000
50%,10718.500,13.000,51.500,21.000,0.000,21.000,1.000
75%,23787.000,22.000,114.750,36.750,26.000,45.750,1.000
max,260543.000,99.000,25992.000,188.000,292.000,93.000,1.000


6활동성 8가지 activity

In [26]:
activity_ft1.describe()

,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,부실판단
count,534.000,534.000,534.000,534.000,534.000,534.000,534.000,534.000,534.000
mean,0.678,7.912,2.022,13.163,0.850,0.680,1.946,1.404,0.504
std,0.427,7.815,1.625,103.168,0.780,0.657,2.979,0.905,0.500
min,0.078,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.390,4.240,1.000,1.000,1.000,0.000,1.000,1.000,0.000
50%,0.587,6.245,2.000,3.000,1.000,1.000,1.000,1.000,1.000
75%,0.887,9.172,3.000,5.000,1.000,1.000,2.000,2.000,1.000
max,3.842,86.680,17.000,2292.000,13.000,9.000,52.000,10.000,1.000


7기타 etc

In [27]:
etc_ft1.describe()

,영업년수,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,534.000,534.000,534.000,534.000,534.000,531.000,534.000,534.000,534.000,534.000,534.000
mean,26.813,0.353,0.223,0.631,2.380,13.183,0.298,0.290,0.223,0.189,0.504
std,13.591,0.486,283.809,2.947,72.371,318.112,0.458,0.454,0.417,0.392,0.500
min,5.000,0.015,-5550.949,-47.302,-244.700,-301.491,0.000,0.000,0.000,0.000,0.000
25%,18.000,0.127,-0.927,0.421,-1.295,-1.876,0.000,0.000,0.000,0.000,0.000
50%,25.000,0.219,1.744,0.832,-0.388,-0.467,0.000,0.000,0.000,0.000,1.000
75%,32.000,0.410,4.151,1.053,0.270,0.221,1.000,1.000,0.000,0.000,1.000
max,107.000,7.116,3404.076,15.017,1529.301,7211.514,1.000,1.000,1.000,1.000,1.000


# 도입정상

In [28]:
도입정상 = listed_X_train[(listed_X_train['도입기'] == 1) & (listed_X_train['부실판단'] == 0)]

도입정상 결측치확인

In [29]:
도입정상.isna().sum().sort_values(ascending=False)

회사명                  0
거래소코드                0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

# 도입부실

In [30]:
도입부실 = listed_X_train[(listed_X_train['도입기'] == 1) & (listed_X_train['부실판단'] == 1)]

도입부실 결측치확인

In [31]:
도입부실.isna().sum().sort_values(ascending=False)

회사명                  0
거래소코드                0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

# 성장정상

In [32]:
성장정상 = listed_X_train[(listed_X_train['성장기'] == 1) & (listed_X_train['부실판단'] == 0)]

성장정사 결측치확인

In [33]:
성장정상.isna().sum().sort_values(ascending=False)

회사명                  0
거래소코드                0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

# 성장부실

In [34]:
성장부실 = listed_X_train[(listed_X_train['성장기'] == 1) & (listed_X_train['부실판단'] == 1)]

성장부실 결측치 확인

In [35]:
성장부실.isna().sum().sort_values(ascending=False)

회사명                  0
거래소코드                0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

# 성숙정상

In [36]:
성숙정상 = listed_X_train[(listed_X_train['성숙기'] == 1) & (listed_X_train['부실판단'] == 0)]

성숙정상 결측치 확인

In [37]:
성숙정상.isna().sum().sort_values(ascending=False)

회사명                  0
거래소코드                0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

# 성숙부실

In [38]:
성숙부실 = listed_X_train[(listed_X_train['성숙기'] == 1) & (listed_X_train['부실판단'] == 1)]

성숙부실 결측치 확인

In [39]:
성숙부실.isna().sum().sort_values(ascending=False)

회사명                  0
거래소코드                0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

# 쇠퇴정상

In [40]:
쇠퇴정상 = listed_X_train[(listed_X_train['쇠퇴기'] == 1) & (listed_X_train['부실판단'] == 0)]

쇠퇴정상 결측치 확인

In [41]:
쇠퇴정상.isna().sum().sort_values(ascending=False)

영업재무현금비율             1
회사명                  0
영업년수                 0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
상장년수                 0
거래소코드                0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

정규성 확인

In [42]:
na_col0 = 쇠퇴정상.isna().sum().sort_values(ascending=False).index[0]
# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴정상[na_col0].dropna()

In [43]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.5316891670227051, p-value: 1.1620824835745225e-07
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [44]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 4.278001584601892
유의수준 15.0%: 임계값 = 0.513
유의수준 10.0%: 임계값 = 0.584
유의수준 5.0%: 임계값 = 0.701
유의수준 2.5%: 임계값 = 0.817
유의수준 1.0%: 임계값 = 0.972
데이터는 정규 분포를 따르지 않습니다.


In [45]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.350764283857121, p-value: 0.0038153471195201405
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


쇠퇴정상 fillna median

In [46]:
쇠퇴정상[na_col0].fillna(data0.median(), inplace=True)

# 쇠퇴부실

In [47]:
쇠퇴부실 = listed_X_train[(listed_X_train['쇠퇴기'] == 1) & (listed_X_train['부실판단'] == 1)]

쇠퇴부실 결측치 확인

In [48]:
쇠퇴부실.isna().sum().sort_values(ascending=False)

영업재무현금비율             2
회사명                  0
영업년수                 0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
상장년수                 0
거래소코드                0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

정규성 확인

In [49]:
na_col0 = 쇠퇴부실.isna().sum().sort_values(ascending=False).index[0]
# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴부실[na_col0].dropna()

In [50]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.2404172420501709, p-value: 1.0671748250122229e-17
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [51]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 18.700642344916645
유의수준 15.0%: 임계값 = 0.549
유의수준 10.0%: 임계값 = 0.625
유의수준 5.0%: 임계값 = 0.75
유의수준 2.5%: 임계값 = 0.875
유의수준 1.0%: 임계값 = 1.041
데이터는 정규 분포를 따르지 않습니다.


In [52]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.20775723577398203, p-value: 0.0027997488783086943
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


쇠퇴부실 fillna median

In [53]:
쇠퇴부실[na_col0].fillna(data0.median(), inplace=True)

# to_csv

In [54]:
filled_train_data = pd.concat([도입부실, 도입정상, 성장부실, 성장정상, 성숙부실, 성숙정상, 쇠퇴부실, 쇠퇴정상], axis=0, ignore_index=True)

train set

In [55]:
filled_train_data.to_csv('../data/datasets/listed_filled_train_data.csv', index=False)

test set

In [56]:
listed_X_test['부실판단'] = listed_y_test
listed_X_test['부실판단']

153   0.000
263   0.000
423   0.000
174   1.000
681   1.000
       ... 
89    0.000
418   1.000
578   1.000
92    0.000
305   1.000
Name: 부실판단, Length: 197, dtype: float64

In [57]:
listed_X_test.to_csv('../data/datasets/listed_test_data.csv', index=False)

In [58]:
listed_X_test

,회사명,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
153,(주)세우글로벌,13000,2022,53,9484,4,8,1104,976,9,...,0.083,2.514,0.659,-0.279,-219196.000,0,1,0,0,0.000
263,(주)엑시콘,92870,2022,-19,-59,-3,-5,351,177,17,...,0.215,2.912,0.632,-4.115,0.697,0,0,0,1,0.000
423,(주)타이거일렉,219130,2022,20,34,5,6,227,148,31,...,0.050,1.928,0.709,-3.478,-3.442,0,1,0,0,0.000
174,(주)스타플렉스,115570,2020,-3,-3,-2,-3,96,61,134,...,0.137,-1.291,0.654,-2.021,0.313,0,0,0,1,1.000
681,피에스텍(주),2230,2020,6,12,1,2,1336,1256,14,...,0.116,-9.796,-0.988,0.529,-0.259,0,1,0,0,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,(주)마니커에프앤지,195500,2022,-7,-11,-3,-3,162,115,60,...,0.121,-5.265,13.265,-5.544,-937.761,0,0,0,1,0.000
418,(주)큐로홀딩스,51780,2014,-1,-1,-1,-3,14,7,138,...,0.264,-50.663,0.018,-2.675,-33.570,0,1,0,0,1.000
578,씨앤티85(주),56730,2014,-6,-6,-4,-21,39,24,279,...,0.332,0.072,0.612,2.012,-0.685,1,0,0,0,1.000
92,(주)매커스,93520,2022,11,3541,7,11,142,119,207,...,0.065,1.470,1.093,-6.005,-3.524,0,1,0,0,0.000
